In [1]:
# An example of intersection of cubics

import time
start_time = time.time()
p = next_prime(2^28)
d=8
n = binomial(d+2,2)-1
F = GF(p)
#F = QQ
P1.<t,v> =ProjectiveSpace(F,1) 
R.<t,v> = PolynomialRing(F,2)
P2 = P1.veronese_embedding(2).codomain()
phi1 = P1.veronese_embedding(2)
phi2 = P2.veronese_embedding(d)

c_1 = (t*v,t^2,v^2)

phi = phi2*phi1
B = phi2.defining_polynomials()
c1_embedded = [B[i](c_1) for i in range(len(B))]

ls = phi.defining_polynomials()
b_bar = vector(set(ls))


M = Matrix(F,len(c1_embedded),len(b_bar))
for i in range(len(B)):
    for j in range(len(b_bar)):
        M[i,j] = c1_embedded[i].coefficient(b_bar[j])
S = Permutations([i+1 for i in range(n+1)])    
V = Permutation(S.random_element()).to_matrix()
alpha = Matrix.random(F,2,algorithm='unimodular').augment(random_matrix(F,2,1)).stack(matrix(F,1,3,[0,0,1]))
v1=vector(alpha*(matrix(R,3,1,[t,v,1])))
u = (t,v,1)
ls2 = [B[i](u) for i in range(len(B))]
ls = []
for i in range(len(B)):
    for j in range(len(ls2)):
        if j<=len(ls2)-2:
            ls.append(B[i](*v1).monomial_coefficient(ls2[j]))
        if j==len(ls2)-1:
            ls.append(B[i](*v1).constant_coefficient())
A = matrix(F,n+1,ls)
N = V*A*M
K = N.kernel()
w1,w2,w3 = [K.matrix()[i] for i in range(3)] # Public Key of Alice

# Bob

beta = Matrix.random(F,2,algorithm='unimodular').augment(random_matrix(F,2,1)).stack(matrix(F,1,3,[0,0,1]))
v2=vector(beta*(matrix(R,3,1,[t,v,1])))
u = (t,v,1)
ls1 = []
for i in range(len(B)):
    for j in range(len(ls2)):
        if j<=len(ls2)-2:
            ls1.append(B[i](*v2).monomial_coefficient(ls2[j]))
        if j==len(ls2)-1:
            ls1.append(B[i](*v2).constant_coefficient())
B1 = matrix(F,n+1,ls1)
#e1= F.random_element()
#e2 = F.random_element()
#B1 = A^e1*A^e2
B2 = V*B1*M
K = B2.kernel()
u1,u2,u3 = [K.matrix()[i] for i in range(3)] # Public Key of Bob


# Alice calculates
a1 = matrix(u1*N)*matrix(b_bar).transpose()
a2 = matrix(u2*N)*matrix(b_bar).transpose()
a3 = matrix(u3*N)*matrix(b_bar).transpose()
a1 = a1[0][0]
a2 = a2[0][0]
a3 = a3[0][0]
f =gcd(a1,a2)
f1 = f.subs(v=t, t=t^2)
f2 = f1/t^4
R1.<x> = PolynomialRing(F)
f2 = f2.subs(t=x)
f_list= f2.coefficients()

# Bob Computes
b1 = matrix(w1*B2)* matrix(b_bar).transpose()
b2 = matrix(w2*B2)*matrix(b_bar).transpose()
b3 = matrix(w3*B2)*matrix(b_bar).transpose()
b1 = b1[0][0]
b2 = b2[0][0]
b3 = b3[0][0]
g = gcd(b1,b2)
g1 = g.subs(v=t,t=t^2)
g2 = g1/t^4
g2= g2.subs(t=x)

    
g_list= g2.coefficients()

R.<x0,x1,x2,x3,sigma0,sigma1,sigma2,sigma3>= PolynomialRing(F,8)    
M = Matrix(R,3,4,[1,1,1,1,x0,x1,x2,x3,x0^2,x1^2,x2^2,x3^2])
ls = M.minors(3)
ls1 = [ls[i]^2 for i in range(4)]
e = SymmetricFunctions(F).e()
ls2 = [e([i]).expand(4,alphabet = 'y') for i in range(1,5)] # Elementary symmetric functions of four variables
ls3 = [ls2[i](*vector(ls1)) for i in range(4)] # Evaluation of the squares in elementary symmetric functions

# Expression in symmetric functions
X= []
for i in range(4):
    Res=[]
    Res.append(ls3[i])
    Res.append(sigma0 - (x0 + x1 + x2 + x3))
    Res.append(sigma1 - (x0*x1 + x0*x2 + x1*x2 + x0*x3 + x1*x3 + x2*x3))
    Res.append(sigma2 - (x0*x1*x2 + x0*x1*x3 + x0*x2*x3 + x1*x2*x3))
    Res.append(sigma3 - (x0*x1*x2*x3))
    I = R.ideal(Res)      
    J = I.groebner_basis()    
    if i ==0:
        X.append(J[1])
    else:
        X.append(J[0])



Alice_invariants = []
for i in range(4):
    Alice_invariants.append(X[i].subs(sigma3= f_list[0],sigma2 = f_list[1],sigma1= f_list[2],sigma0 = f_list[3]))
    
    
Bob_invariants = []
for i in range(4):
    Bob_invariants.append(X[i].subs(sigma3= g_list[0],sigma2 = g_list[1],sigma1= g_list[2],sigma0 = g_list[3]))


print( "Invaiants of Alice is ={} \n and Invariants of Bob is = {}".format(Alice_invariants,Bob_invariants))
end_time = time.time()
print(" Time taken  {} seconds".format(abs(end_time - start_time))) 

Invaiants of Alice is =[8497421, 117917862, -5532885, 40842669] 
 and Invariants of Bob is = [8497421, 117917862, -5532885, 40842669]
 Time taken  1.1821939945220947 seconds
